In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none"
pd.set_option('display.max_columns', None)

### I will create the interaction matrix from history data and save it in order to quick access

In [15]:
size = 'demo'
type_ = 'validation'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/{type_}/history.parquet'

history_df = pd.read_parquet(history_train_path)
print(history_df.shape)
try:
    history_df.drop('impression_time_fixed', axis=1, inplace=True) # i will drop this column initialy even though i can proccess it and use it
except:
    pass

behaviors_path = f'./files/parquet/ebnerd_{size}/{type_}/behaviors.parquet'
behaviors_df = pd.read_parquet(behaviors_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_df.shape)


# from sklearn.model_selection import train_test_split

# val_df, test_df = train_test_split(history_df, test_size=0.3, random_state=42)

# # Save the splits as Parquet files
# test_path = "./files/parquet/ebnerd_demo/test/history.parquet"
# val_path = "./files/parquet/ebnerd_demo/validation/history.parquet"

# test_df.to_parquet(test_path, index=False)
# val_df.to_parquet(val_path, index=False)

def replace_nan_with_zero_in_column(df, column_name):
    """
    Function to replace NaN values in a specific column with 0.
    
    Parameters:
    - df: Pandas DataFrame
    - column_name: Name of the column in which to replace NaN values with 0
    
    Returns:
    - df: Updated DataFrame with NaN values replaced by 0 in the specified column
    """
    df[column_name].fillna(0, inplace=True)
    return df

# Apply the function to replace NaN with 0 in 'read_time_fixed' column
behaviors_df = replace_nan_with_zero_in_column(behaviors_df, 'scroll_percentage')
behaviors_df = replace_nan_with_zero_in_column(behaviors_df, 'read_time')

# def replace_nans_with_empty_lists(df, column_names):
#     """
#     Replaces NaN values in specified DataFrame columns with empty lists.

#     Parameters:
#     df (pd.DataFrame): The DataFrame containing the columns.
#     column_names (list of str): The names of the columns to process.

#     Returns:
#     pd.DataFrame: The DataFrame with NaNs replaced by empty lists in the specified columns.
#     """
#     for column in column_names:
#         def replace_nan(x):
#             if isinstance(x, (list, np.ndarray)):
#                 # If x is a list or array, check for any NaNs within
#                 if any(pd.isnull(e) for e in x):
#                     return []
#                 else:
#                     return x
#             else:
#                 # If x is not a list or array, check if it is NaN
#                 return [] if pd.isnull(x) else x
        
#         df[column] = df[column].apply(replace_nan)
#     return df

# columns_to_replace = ['article_ids_clicked', 'read_time', 'scroll_percentage'] #
# final_df = replace_nans_with_empty_lists(final_df, columns_to_replace)

# Preprocess behaviour
behaviors_df = behaviors_df[['article_ids_clicked','user_id','read_time','scroll_percentage']]
behaviors_df = behaviors_df.explode('article_ids_clicked')

# behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()

# behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index()

behaviors_df_merged= pd.merge(behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index(),behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index(),on='user_id')

behaviors_df= pd.merge(behaviors_df_merged,behaviors_df.groupby('user_id')['scroll_percentage'].apply(list).reset_index(),on='user_id')

behaviors_df.head()


(1093, 5)
Raw demo validation behaviors df shape:           (17749, 17)


,user_id,article_ids_clicked,read_time,scroll_percentage
0,19181,"[9787863, 9783164, 9786359, 9780547, 9787593, ...","[18.0, 5.0, 16.0, 26.0, 46.0, 79.0, 55.0, 15.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,21271,"[9785668, 9786172, 9786222, 9782407, 9785113]","[10.0, 25.0, 15.0, 16.0, 23.0]","[100.0, 0.0, 0.0, 0.0, 0.0]"
2,21774,[9785835],[47.0],[0.0]
3,22779,"[9784044, 9786209, 9784852, 9780702, 9787261]","[13.0, 61.0, 58.0, 11.0, 14.0]","[0.0, 0.0, 100.0, 0.0, 0.0]"
4,22895,"[9781013, 9790475, 9781709, 9782361, 9782421, ...","[17.0, 29.0, 10.0, 8.0, 7.0, 20.0, 16.0, 7.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [16]:
## Merge behaviours and history-left join
final_df = pd.merge(history_df,behaviors_df,on='user_id',how='outer')

def convert_arrays_to_lists(df, column_name):
    """
    Function to convert arrays (or other iterable objects) in a column to lists.
    
    Parameters:
    - df: Pandas DataFrame
    - column_name: Name of the column containing arrays or other iterables
    
    Returns:
    - df: Updated DataFrame with the column converted to lists
    """
    df[column_name] = df[column_name].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
    return df

# Convert arrays in 'data_arrays' column to lists
for col in final_df.columns: #['scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed']
    final_df = convert_arrays_to_lists(final_df, col)

def replace_nans_with_empty_lists(df, column_names):
    """
    Replaces NaN values in specified DataFrame columns with empty lists.

    Args:
    df (pd.DataFrame): The input DataFrame.
    column_names (list of str): The column names in which to replace NaNs with empty lists.

    Returns:
    pd.DataFrame: The DataFrame with NaNs replaced by empty lists in the specified columns.
    """
    for column in column_names:
        df[column] = df[column].apply(lambda x: x if isinstance(x, list) else [])
    return df
# Apply the function to replace NaN with 0 in 'read_time_fixed' column
columns_to_replace = ['article_ids_clicked', 'read_time', 'scroll_percentage']

final_df = replace_nans_with_empty_lists(final_df, columns_to_replace)

final_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_clicked,read_time,scroll_percentage
0,2515364,"[86.0, 92.0, 100.0, 100.0, 100.0, 100.0, 36.0,...","[9763559, 9764444, 9764759, 9749628, 9749628, ...","[52.0, 23.0, 87.0, 100.0, 2.0, 48.0, 13.0, 5.0...",[],[],[]
1,299355,"[76.0, 48.0, 28.0, 37.0, 35.0, 71.0, nan, 72.0...","[9750802, 9750815, 9750696, 9748321, 9747411, ...","[10.0, 2.0, 2.0, 3.0, 6.0, 2.0, 0.0, 1.0, 2.0,...","[9784275, 9788404, 9787525, 9783852, 9785751, ...","[5.0, 12.0, 17.0, 17.0, 5.0, 7.0, 29.0, 12.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,551337,"[19.0, 11.0, 100.0, 16.0, 10.0, 100.0, nan, 23...","[9754603, 9753748, 9753207, 9755964, 9755730, ...","[5.0, 4.0, 34.0, 3.0, 4.0, 69.0, 0.0, 1.0, 23....","[9779225, 9787811, 9777016, 9781520, 9782806, ...","[33.0, 29.0, 27.0, 20.0, 21.0, 14.0, 22.0, 28....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1520051,"[19.0, 29.0, 100.0, 28.0, 23.0, 100.0, 100.0, ...","[9775804, 9776369, 9776508, 9776434, 9774559, ...","[10.0, 11.0, 68.0, 3.0, 13.0, 45.0, 67.0, 3.0,...","[9786693, 9781763, 9784444, 9775978, 9780925, ...","[33.0, 58.0, 14.0, 33.0, 7.0, 19.0, 26.0, 25.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,251987,"[100.0, 100.0, 94.0, 100.0, 85.0, 100.0, 32.0,...","[9746342, 9748450, 9750802, 9748450, 9750829, ...","[12.0, 41.0, 76.0, 4.0, 44.0, 10.0, 9.0, 5.0, ...","[9783628, 9789644, 9783404, 9780849, 9779807, ...","[32.0, 10.0, 16.0, 8.0, 9.0, 10.0, 6.0, 88.0, ...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100..."


In [17]:
def extend_lists_in_columns(df, col1, col2, new_col_name):
    """
    Extends the list in each cell of the second column with the list in the first column
    and stores the result in a new column.

    Args:
    df (pd.DataFrame): The input DataFrame.
    col1 (str): The name of the first column containing lists.
    col2 (str): The name of the second column containing lists.
    new_col_name (str): The name of the new column to store the extended lists.

    Returns:
    pd.DataFrame: The DataFrame with the new column containing the extended lists.
    """
    def extend_lists(row):
        list1 = row[col1] if isinstance(row[col1], list) else []
        list2 = row[col2] if isinstance(row[col2], list) else []
        return list1 + list2

    df[new_col_name] = df.apply(extend_lists, axis=1)
    return df
    

# Combine article_id_fixed and article_ids_clicked into a single column
final_df = extend_lists_in_columns(final_df, 'article_id_fixed', 'article_ids_clicked', 'merged_article_ids')
final_df = extend_lists_in_columns(final_df, 'scroll_percentage_fixed', 'scroll_percentage', 'merged_scroll_percentage')
final_df = extend_lists_in_columns(final_df, 'read_time_fixed', 'read_time', 'merged_read_time')

final_df = final_df[['merged_article_ids','merged_read_time','user_id']]
final_df = final_df.explode('merged_article_ids')
print(final_df.shape)
final_df.head(2)

(187407, 3)


,merged_article_ids,merged_read_time,user_id
0,9763559,"[52.0, 23.0, 87.0, 100.0, 2.0, 48.0, 13.0, 5.0...",2515364
0,9764444,"[52.0, 23.0, 87.0, 100.0, 2.0, 48.0, 13.0, 5.0...",2515364


In [26]:
# final_df.set_index('user_id',inplace=True)
# final_df = final_df.explode(['merge_scroll_percentage','merge_article_ids']).dropna(subset='merge_scroll_percentage')
# final_df.drop(axis=1,inplace=True,columns='merge_scroll_percentage')

In [18]:
# Creation of interaction matrix

interaction_matrix = pd.DataFrame(columns=final_df.columns)
counter = 0
for user_id in tqdm(np.unique(final_df.index.values).tolist()):
    
    temp_df = final_df[final_df.index == user_id].copy()
    temp_df = temp_df.explode('merged_article_ids')
    temp_df.set_index('user_id',inplace=True)
    temp_lst = []

    # Fix the lists that exist in each cell and keep the values that i care
    temp_counter = 0
    for lst in list(temp_df['merged_read_time']):
        temp_lst.append(lst[temp_counter])
        temp_counter += 1
    
    temp_df['merged_read_time_fixed'] = temp_lst

    # Drop first unneccesary column
    temp_df.drop('merged_read_time', axis=1, inplace= True)
    temp_pivoted_df = temp_df.pivot_table(index = temp_df.index, columns='merged_article_ids', values='merged_read_time_fixed', aggfunc='sum').copy()

    if counter == 0:
        interaction_matrix = temp_pivoted_df.copy()
        #print(interaction_matrix)
    else:
        interaction_matrix = pd.concat([interaction_matrix, temp_pivoted_df], axis=0, ignore_index=False)
    
    counter += 1

C:\Users\Intel PC G5900\AppData\Local\Temp\ipykernel_6452\3818264614.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id in tqdm(np.unique(final_df.index.values).tolist()):


  0%|          | 0/1539 [00:00<?, ?it/s]

In [19]:
interaction_matrix.shape

(1539, 4545)

In [20]:
interaction_matrix = interaction_matrix.fillna(0)

interaction_matrix.head()

merged_article_ids  9749628  9762114  9763559  9764444  9764608  9764759  \
user_id                                                                    
2515364               102.0     13.0     52.0     23.0     27.0     87.0   
299355                  0.0      0.0      0.0      0.0      0.0      0.0   
551337                  0.0      0.0      0.0      0.0      0.0      0.0   
1520051                 0.0      0.0      0.0      0.0      0.0      0.0   
251987                  0.0    184.0      0.0      0.0     24.0    129.0   

merged_article_ids  9765753  9765839  9765894  9765965  9766599  9767554  \
user_id                                                                    
2515364               150.0      6.0     53.0    293.0      7.0     79.0   
299355                  0.0      0.0      0.0      0.0      0.0      0.0   
551337                  0.0      0.0      0.0      0.0      0.0      0.0   
1520051                 0.0      0.0      0.0      0.0      0.0      0.0   
251987                  0.0      0.0      0.0      0.0      0.0      0.0   

merged_article_ids  9767741  9768311  9768377  9768387  9771627  9772050  \
user_id                                                                    
2515364               198.0      9.0      7.0     16.0     19.0     45.0   
299355                  0.0      0.0      0.0      0.0      0.0      0.0   
551337                  0.0      0.0      0.0      0.0      0.0      0.0   
1520051                 0.0      0.0      0.0      0.0      0.0      0.0   
251987                  0.0      0.0      0.0      0.0      0.0      0.0   

merged_article_ids  9772366  9772380  9772442  9772517  9777319  9777374  \
user_id                                                                    
2515364                12.0      9.0     67.0     24.0     32.0      5.0   
299355                  0.0      6.0      0.0      2.0      0.0      0.0   
551337                  8.0      0.0      0.0      0.0      0.0      1.0   
1520051                 0.0      0.0      0.0      0.0      0.0      0.0   
251987                  0.0      0.0      0.0      0.0      0.0      0.0   

merged_article_ids  9777648  9779511  9509856  9716537  9744070  9744403  \
user_id                                                                    
2515364                11.0    108.0      0.0      0.0      0.0      0.0   
299355                  0.0      0.0     11.0      5.0      1.0     13.0   
551337                  0.0      0.0      0.0      6.0      0.0      0.0   
1520051                 0.0     50.0      0.0      0.0      0.0      0.0   
251987                  0.0      0.0      0.0      0.0      0.0      0.0   

merged_article_ids  9747411  9747684  9748321  9750696  9750802  9750815  \
user_id                                                                    
2515364                 0.0      0.0      0.0      0.0      0.0      0.0   
299355                  6.0    130.0      3.0      2.0     10.0      2.0   
551337                  0.0      0.0      0.0      0.0      0.0      0.0   
1520051                 0.0      0.0      0.0      0.0      0.0      0.0   
251987                  0.0    455.0      0.0      0.0     76.0      9.0   

merged_article_ids  9751064  9751531  9754025  9754121  9754122  9754361  \
user_id                                                                    
2515364                 0.0      0.0      0.0      0.0      0.0      0.0   
299355                  2.0      0.0     64.0    346.0     75.0      2.0   
551337                  0.0      0.0      0.0      0.0      0.0      0.0   
1520051                 0.0      0.0      0.0      0.0      0.0      0.0   
251987                  0.0      0.0      0.0     26.0      0.0      0.0   

merged_article_ids  9754388  9754598  9754794  9755010  9755098  9755181  \
user_id                                                                    
2515364                 0.0      0.0      0.0      0.0      0.0      0.0   
299355                 50.0     23.0   10

In [21]:
fillna_value = '0'
# Save the interaction matrix to pickle
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'

interaction_matrix.to_pickle(interaction_matrix_file_path)

In [200]:
# Read interaction matrix pickle file
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)

FileNotFoundError: [Errno 2] No such file or directory: './files/pickle/interaction_matrix_demo_0.pkl'

## Other

In [ ]:
# # Combine interactions into a single metric
# def combine_interactions(interactions, interactions2, w1=1, w2=1):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# # Flatten the interaction data with combined interactions
# flattened_data = []

# for _, row in history_df.iterrows():
#     user_id = row['user_id']
#     article_ids = row['article_id_fixed']
#     combined_interactions = combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed'])
    
#     for article_id, interaction_combined in zip(article_ids, combined_interactions):
#         flattened_data.append({
#             'user_id': user_id, 
#             'article_id_fixed': article_id, 
#             'interaction_combined': interaction_combined
#         })

# # Convert to DataFrame
# interaction_df = pd.DataFrame(flattened_data)